# Load Configurations & Libraries

In [ ]:
# If we run this machine on GPU machine it will be faster.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
apipath = r'H:\\My Drive\\config\\hbqa.txt'
apipath = r'/content/drive/MyDrive/config/hbqa-colab.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)

datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']
corpus_sectionpath = config['global']['CORPUS_FOLDER_SECTIONS']
OPENAI_KEY = config['global']['OPENAI_KEY']
PINECONE_KEY = config['global']['PINECONE_KEY']
PINECONE_ENV = config['global']['PINECONE_ENV']
CHATPDF_KEY = config['global']['CHATPDF_KEY']

# PINECONE is Vector Database. To store the vector so that we can quickly search the vector space.
# https://app.pinecone.io
# get PINECONE_API_KEY key from app.pinecone.io
# find your PINECONE_ENVIRONMENT next to the api key in pinecone console

### Load Embedding Model. It will create 384 dim vector for the sentences.

In [ ]:
!pip install -Uq sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

### Selecting and Loading Embedding Model.
- Different models are created on different kind of corpus
- Different vector size
- Some are normailzied, in that case dot product and cosine is same. Dot is less costly

In [4]:
# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
##290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
##1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

# embmodelname = 'multi-qa-distilbert-cos-v1'
# embmodelname = 'all-MiniLM-L6-v2'
# embmodelname = 'all-distilroberta-v1'
embmodelname = 'multi-qa-mpnet-base-dot-v1'
# embmodelname =  'all-roberta-large-v1'

embmodelname1 = "_" + embmodelname

In [5]:
embmodelname

'multi-qa-mpnet-base-dot-v1'

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

In [ ]:
#util.dot_score, util.cos_sim

embmodel = SentenceTransformer(embmodelname)

# Predict Document Id from Question

In [ ]:
df_qa = pd.read_csv(datapath + '06-HBQA_Manual_with_Chunk.csv')

In [ ]:
print(df_qa.shape)
df_qa.head(2)


(1102, 11)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [ ]:
# import pinecone
# # index_name = 'hbqa'
# # now connect to the index
# index = pinecone.Index(index_name)

In [ ]:
df_hbqa_chunk = pd.read_csv(datapath + '07.1-HBQA_Chunk_Vector'+embmodelname1+'.csv')
(df_hbqa_chunk.shape)
df_hbqa_chunk.head(3)


,index,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,ChunkVector_multi-qa-mpnet-base-dot-v1
0,222,Book03_002,389,4793,808,1075,Even this is eternal morality. They that perfo...,"Write 5 unique questions, corresponding answer...",Question 1: What is the significance of perfor...,"[-0.044648099690675735, -0.0822770893573761, -..."
1,223,Book03_003,390,5453,851,1132,"Section III\n""Vaisampayana said, 'Yudhishthira...","Write 6 unique questions, corresponding answer...",Question 1: What predicament does Yudhishthira...,"[0.2307928055524826, -0.18484173715114594, -0...."
2,224,Book03_004,392,4383,775,1031,"Section IV\n""Vaisampayana said,--""After the Pa...","Write 5 unique questions, corresponding answer...",Question 1: What was the primary concern expre...,"[0.10455890744924545, -0.17260190844535828, -0..."


In [ ]:
embmodelname1

'_multi-qa-mpnet-base-dot-v1'

In [ ]:
import ast
len(ast.literal_eval(df_hbqa_chunk['ChunkVector'+embmodelname1][10]))

768

In [ ]:
chunk_id = df_hbqa_chunk['Chunk_Id'].to_list()

chunk_vector = [ast.literal_eval(df_hbqa_chunk['ChunkVector'+embmodelname1][i]) for i in range(df_hbqa_chunk.shape[0])]

In [ ]:
len(chunk_vector), type(chunk_vector), len(chunk_vector[0])

(334, list, 768)

In [ ]:
chunk_vector = torch.tensor(chunk_vector)

In [ ]:
len(chunk_vector), type(chunk_vector), len(chunk_vector[0])

(334, torch.Tensor, 768)

In [ ]:
# for i in df_qa.index:
df_pred = pd.DataFrame(columns = ['Ques_Id','Chunk_Id','Pred_Chunk_Id'])

for i in df_qa.index:
  query = df_qa.loc[i,'Question']
  question_embedding = torch.tensor(embmodel.encode(query).tolist())
  top_k=10
  hits = util.semantic_search(question_embedding, chunk_vector, top_k=top_k)

  predictedId=[]
  for hit in hits[0]:
    hbqa_chunk_idx = hit['corpus_id']
    # print('Id:',hbqa_chunk_idx, 'Chunk_Id: ', df_hbqa_chunk.loc[hbqa_chunk_idx,'Chunk_Id'], ' Score:', hit['score'])
    predictedId.append(df_hbqa_chunk.loc[hbqa_chunk_idx,'Chunk_Id'])

  # Save prediction in dataframe
  df_pred.loc[i]= (df_qa.loc[i,'Ques_Id'], df_qa.loc[i,'Chunk_Id'], predictedId)
  print('Predicting: ',i)
  # break



Predicting:  0
Predicting:  1
Predicting:  2
Predicting:  3
Predicting:  4
Predicting:  5
Predicting:  6
Predicting:  7
Predicting:  8
Predicting:  9
Predicting:  10
Predicting:  11
Predicting:  12
Predicting:  13
Predicting:  14
Predicting:  15
Predicting:  16
Predicting:  17
Predicting:  18
Predicting:  19
Predicting:  20
Predicting:  21
Predicting:  22
Predicting:  23
Predicting:  24
Predicting:  25
Predicting:  26
Predicting:  27
Predicting:  28
Predicting:  29
Predicting:  30
Predicting:  31
Predicting:  32
Predicting:  33
Predicting:  34
Predicting:  35
Predicting:  36
Predicting:  37
Predicting:  38
Predicting:  39
Predicting:  40
Predicting:  41
Predicting:  42
Predicting:  43
Predicting:  44
Predicting:  45
Predicting:  46
Predicting:  47
Predicting:  48
Predicting:  49
Predicting:  50
Predicting:  51
Predicting:  52
Predicting:  53
Predicting:  54
Predicting:  55
Predicting:  56
Predicting:  57
Predicting:  58
Predicting:  59
Predicting:  60
Predicting:  61
Predicting:  62
Pr

In [ ]:
df_pred.to_csv(datapath + '08.1-Predict-DocumentId-for-Ques'+embmodelname1+'.csv', index=False)

In [6]:
df_pred = pd.read_csv(datapath + '08.1-Predict-DocumentId-for-Ques'+embmodelname1+'.csv')

In [8]:
import ast

In [10]:
def check_id_in_1pred(row):
    # return int('id' + str(row['Chunk_Id']) in row['Pred_Chunk_Id'][0:1])
    #  return row['Chunk_Id'] in row['Pred_Chunk_Id'][0:1]
     return row['Chunk_Id'] in ast.literal_eval(row['Pred_Chunk_Id'])[0:1]


def check_id_in_2pred(row):
    # return int('id' + str(row['Chunk_Id']) in row['Pred_Chunk_Id'][0:2])
    #  return row['Chunk_Id'] in row['Pred_Chunk_Id'][0:2]
    return row['Chunk_Id'] in ast.literal_eval(row['Pred_Chunk_Id'])[0:2]

def check_id_in_3pred(row):
    # return int('id' + str(row['Chunk_Id']) in row['Pred_Chunk_Id'][0:3])
    # return row['Chunk_Id'] in row['Pred_Chunk_Id'][0:3]
    return row['Chunk_Id'] in ast.literal_eval(row['Pred_Chunk_Id'])[0:3]

def check_id_in_4pred(row):
    # return int('id' + str(row['Chunk_Id']) in row['Pred_Chunk_Id'][0:4])
    #  return row['Chunk_Id'] in row['Pred_Chunk_Id'][0:4]
     return row['Chunk_Id'] in ast.literal_eval(row['Pred_Chunk_Id'])[0:4]

def check_id_in_5pred(row):
    # return int('id' + str(row['Chunk_Id']) in row['Pred_Chunk_Id'][0:5])
    #  return row['Chunk_Id'] in row['Pred_Chunk_Id'][0:5]
    return row['Chunk_Id'] in ast.literal_eval(row['Pred_Chunk_Id'])[0:5]

def check_id_in_10pred(row):
    # return int('id' + str(row['Chunk_Id']) in row['Pred_Chunk_Id'])
    #  return row['Chunk_Id'] in row['Pred_Chunk_Id']
    return row['Chunk_Id'] in ast.literal_eval(row['Pred_Chunk_Id'])


# Create a new column 'MRR' based on the condition
df_pred['FoundIn@1'] = df_pred.apply(check_id_in_1pred, axis=1)
df_pred['FoundIn@2'] = df_pred.apply(check_id_in_2pred, axis=1)
df_pred['FoundIn@3'] = df_pred.apply(check_id_in_3pred, axis=1)
df_pred['FoundIn@4'] = df_pred.apply(check_id_in_4pred, axis=1)
df_pred['FoundIn@5'] = df_pred.apply(check_id_in_5pred, axis=1)
df_pred['FoundIn@10'] = df_pred.apply(check_id_in_10pred, axis=1)


In [11]:
df_pred.sample(5)

,Ques_Id,Chunk_Id,Pred_Chunk_Id,FoundIn@1,FoundIn@2,FoundIn@3,FoundIn@4,FoundIn@5,FoundIn@10
1094,1094,935,"[935, 945, 394, 896, 690, 949, 718, 695, 946, ...",True,True,True,True,True,True
147,147,454,"[455, 454, 457, 461, 465, 471, 459, 453, 482, ...",False,True,True,True,True,True
556,556,595,"[597, 595, 599, 598, 593, 516, 436, 432, 596, ...",False,True,True,True,True,True
767,767,705,"[706, 705, 691, 697, 704, 712, 689, 396, 695, ...",False,True,True,True,True,True
465,465,554,"[552, 515, 591, 431, 602, 484, 786, 587, 601, ...",False,False,False,False,False,False


In [12]:
df_pred[['FoundIn@1','FoundIn@2',	'FoundIn@3','FoundIn@4',	'FoundIn@5',	'FoundIn@10']].mean()

FoundIn@1     0.431034
FoundIn@2     0.569873
FoundIn@3     0.629764
FoundIn@4     0.685118
FoundIn@5     0.718693
FoundIn@10    0.805808
dtype: float64

In [13]:
# Number of relevant items in the top K/K

df_pred['P@1'] = df_pred['FoundIn@1']
df_pred['P@2'] = (df_pred['FoundIn@2']/2).apply(lambda x: round(x, 2))
df_pred['P@3'] = (df_pred['FoundIn@3']/3).apply(lambda x: round(x, 2))
df_pred['P@4'] = (df_pred['FoundIn@3']/4).apply(lambda x: round(x, 2))
df_pred['P@5'] = (df_pred['FoundIn@5']/5).apply(lambda x: round(x, 2))
df_pred['P@10'] = (df_pred['FoundIn@10']/10).apply(lambda x: round(x, 2))


In [14]:
# Number of Relevant Items in the Top K/ Total Number of Relevant Items in the Collection

df_pred['R@1'] = df_pred['FoundIn@1']/1
df_pred['R@2'] = df_pred['FoundIn@2']/1
df_pred['R@3'] = df_pred['FoundIn@3']/1
df_pred['R@4'] = df_pred['FoundIn@4']/1
df_pred['R@5'] = df_pred['FoundIn@5']/1
df_pred['R@10'] = df_pred['FoundIn@10']/1

In [ ]:
df_pred['F1@1']   = ( (2*df_pred['P@1']*df_pred['R@1'])/ ( df_pred['P@1']+df_pred['R@1'])).apply(lambda x: round(x, 2)).fillna(0)
df_pred['F1@2']   = ( (2*df_pred['P@2']*df_pred['R@2'])/ ( df_pred['P@2']+df_pred['R@2'])).apply(lambda x: round(x, 2)).fillna(0)
df_pred['F1@3']   = ( (2*df_pred['P@3']*df_pred['R@3'])/ ( df_pred['P@3']+df_pred['R@3'])).apply(lambda x: round(x, 2)).fillna(0)
df_pred['F1@4']   = ( (2*df_pred['P@4']*df_pred['R@4'])/ ( df_pred['P@4']+df_pred['R@4'])).apply(lambda x: round(x, 2)).fillna(0)
df_pred['F1@5']   = ( (2*df_pred['P@5']*df_pred['R@5'])/ ( df_pred['P@5']+df_pred['R@5'])).apply(lambda x: round(x, 2)).fillna(0)
df_pred['F1@10'] =  ( (2*df_pred['P@10']*df_pred['R@10'])/ ( df_pred['P@10']+df_pred['R@10'])).apply(lambda x: round(x, 2)).fillna(0)

In [ ]:
# Mean Average Precision (MAP) is a metric used to evaluate the performance of information retrieval systems,
# particularly in scenarios where there are multiple queries and each query can have multiple relevant documents.
# It's a measure of how well a retrieval system ranks and retrieves relevant documents across different queries.
def calc_MAP(row):
  return round((row['P@1']+row['P@2']+row['P@3']+row['P@4']+row['P@5'])/5,2)

df_pred['MAP'] = df_pred.apply(calc_MAP, axis=1)


In [ ]:
# Mean Reciprocal Rank (MRR) is a metric commonly used to evaluate the performance of ranking-based retrieval systems,
# such as search engines. MRR assesses how well a retrieval system ranks relevant items by considering the position of the
# first relevant item in the ranked list. It's calculated as the average of the reciprocal ranks across a set of queries.

def getReciprocalRank(row):
  K=1 # Number of relevent document
  try:
    # i = row['Pred_Chunk_Id'].index('id'+str(row['Chunk_Id']) )
    i = row['Pred_Chunk_Id'].index(row['Chunk_Id'] )
  except:
    i=-1

  if i==-1:
    RR=0
  else:
    RR = round(1/(i + 1),2)
  return RR

df_pred['RR'] = df_pred.apply(getReciprocalRank, axis=1)

In [ ]:
df_pred[['Ques_Id',
    'P@1', 'P@2', 'P@3', 'P@4', 'P@5', 'P@10',
    'R@1', 'R@2', 'R@3', 'R@4', 'R@5', 'R@10',
    'F1@1', 'F1@2', 'F1@3', 'F1@4', 'F1@5', 'F1@10',
    'MAP', 'RR',]]

,Ques_Id,P@1,P@2,P@3,P@4,P@5,P@10,R@1,R@2,R@3,...,R@5,R@10,F1@1,F1@2,F1@3,F1@4,F1@5,F1@10,MAP,RR
0,0,True,0.5,0.33,0.25,0.2,0.1,1.0,1.0,1.0,...,1.0,1.0,1.0,0.67,0.5,0.4,0.33,0.18,0.46,1.00
1,1,False,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00
2,2,False,0.0,0.00,0.00,0.0,0.1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.00,0.0,0.0,0.00,0.18,0.00,0.14
3,3,False,0.0,0.00,0.00,0.0,0.1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.00,0.0,0.0,0.00,0.18,0.00,0.10
4,4,False,0.0,0.00,0.00,0.2,0.1,0.0,0.0,0.0,...,1.0,1.0,0.0,0.00,0.0,0.0,0.33,0.18,0.04,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1097,True,0.5,0.33,0.25,0.2,0.1,1.0,1.0,1.0,...,1.0,1.0,1.0,0.67,0.5,0.4,0.33,0.18,0.46,1.00
1098,1098,True,0.5,0.33,0.25,0.2,0.1,1.0,1.0,1.0,...,1.0,1.0,1.0,0.67,0.5,0.4,0.33,0.18,0.46,1.00
1099,1099,False,0.5,0.33,0.25,0.2,0.1,0.0,1.0,1.0,...,1.0,1.0,0.0,0.67,0.5,0.4,0.33,0.18,0.26,0.50
1100,1100,True,0.5,0.33,0.25,0.2,0.1,1.0,1.0,1.0,...,1.0,1.0,1.0,0.67,0.5,0.4,0.33,0.18,0.46,1.00


In [ ]:
df_pred[[
    'P@1', 'P@2', 'P@3', 'P@4', 'P@5', 'P@10',
    'R@1', 'R@2', 'R@3', 'R@4', 'R@5', 'R@10',
    'F1@1', 'F1@2', 'F1@3', 'F1@4', 'F1@5', 'F1@10',
    'MAP', 'RR',]].mean()

P@1      0.362976
P@2      0.235481
P@3      0.176679
P@4      0.133848
P@5      0.121779
P@10     0.070962
R@1      0.362976
R@2      0.470962
R@3      0.535390
R@4      0.569873
R@5      0.608893
R@10     0.709619
F1@1     0.362976
F1@2     0.315544
F1@3     0.267695
F1@4     0.214156
F1@5     0.200935
F1@10    0.127731
MAP      0.208294
RR       0.468122
dtype: float64

In [ ]:
df_pred.to_csv(datapath + '08.2-Predict-DocumentId-for-Ques-Metrics'+embmodelname1+'.csv')